<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/4_KNN_y_Arboles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🧑‍🤝‍🧑KNN, 🌳 Árboles de Decisión y 🎛️ Búsqueda de Hiperparámetros

**Objetivos de Aprendizaje**

Al finalizar este notebook, los estudiantes serán capaces de:

1.  **Entender la intuición** detrás de dos modelos de clasificación no paramétricos: **K-Nearest Neighbors (KNN)** y **Árboles de Decisión**.
2. **Comprender el concepto de hiperparámetro** y la necesidad de optimizarlo.
3.  **Aplicar** los conceptos de **Validación Cruzada** (como técnica de evaluación robusta) y **Grid Search** (como técnica de optimización de hiperparámetros).
4.  **Extraer y comparar** la importancia de las variables (`feature importance`) de ambos modelos.

**Introducción**

Imaginemos que somos analistas de riesgo en un banco. Nuestra tarea es decidir si aprobamos o no una nueva tarjeta de crédito a un solicitante. ¿Qué información usaríamos? Probablemente miraríamos su nivel de ingresos, su historial de pagos, su edad, su nivel educativo, etc.

Intuitivamente, haríamos dos cosas:

1.  **Buscar clientes similares:** Compararíamos al nuevo solicitante con clientes que ya tenemos. Si se parece mucho a un grupo de clientes que siempre pagan a tiempo, es probable que él también sea un buen cliente. Esta es la lógica detrás de **KNN**.
2.  **Crear reglas de decisión:** Podríamos establecer una serie de preguntas. Por ejemplo: "¿El solicitante tiene un historial de pagos sin retrasos? **Si es así**, ¿su límite de crédito solicitado es menor a sus ingresos mensuales? **Si es así**, APROBAR". Este proceso secuencial de preguntas es la base de los **Árboles de Decisión**.

Pero, ¿cuántos "vecinos similares" debemos considerar? ¿Qué tan "profundas" o complejas deben ser nuestras reglas? Estas "perillas" que ajustamos en nuestros modelos se llaman **hiperparámetros**. Elegir el valor correcto para estas perillas es crucial, y para ello usaremos una técnica fundamental: **Grid Search**.

**IMPORTANTE:**

- KNN y árboles de decisión son modelos no paramétricos, ya que no asumen ninguna forma funcional (a diferencia de Regresión Lineal donde se asume una línea recta). La estructura del modelo es flexible y se adapta completamente a los datos que observa y la complejidad del modelo no es fija, sino que crece a medida que se agregan más datos.

- Ambos resuelven tanto problemas de regresión como de clasificación (vamos a ver un ejemplo de regresión y en el taller van a trabajar un ejemplo de clasificación).

## 1. Preparación del Entorno y Datos

### 1.1. Carga de Librerías

Importamos las herramientas necesarias. Noten que usamos las versiones `Regressor` de los modelos y métricas de regresión.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor  # <--- Versión de Regresión
from sklearn.tree import DecisionTreeRegressor, plot_tree # <--- Versión de Regresión
from sklearn.metrics import mean_squared_error, r2_score # <--- Métricas de Regresión
from sklearn.inspection import permutation_importance # <--- Para Feature Importance

### Mejorar visualización de dataframes y gráficos

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2. Carga y Exploración de Datos

**Contexto del Caso: Prediciendo la Demanda de Bicicletas Compartidas**

Somos consultores para la ciudad de Seúl. El gobierno quiere optimizar el sistema de bicicletas compartidas. Para reubicar las bicicletas eficientemente y planificar el mantenimiento, necesitan un modelo que prediga la **demanda de bicicletas (cuántas se alquilarán)** en una hora determinada.

Tenemos un dataset con datos históricos por hora que incluye:
* **Información temporal:** Fecha, Hora.
* **Información climática:** Temperatura, Humedad, Velocidad del Viento, Lluvia, Nieve, etc.
* **Contexto del día:** Si es festivo o un día laborable.

**Nuestro Objetivo:** Predecir la variable `Rented Bike Count` (un número continuo). A diferencia de la clase anterior (predecir una categoría sí/no), este es un **problema de regresión**.

**Variables:**

- Date: Fecha
- Rented Bike Count: Número de bicicletas alquiladas  --> **Esta es la variable objetivo**
- Hour: Hora
- Temperature(°C): Temperatura
- Humidity(%): Humedad
- Wind speed (m/s): Velocidad del viento
- Visibility (10m): Visibilidad
- Dew point temperature(°C): Temperatura del punto de rocío
- Solar Radiation (MJ/m2): Radiación solar
- Rainfall(mm): Lluvia
- Snowfall (cm): Nevada
- Seasons: Estaciones
- Holiday: Día festivo
- Functioning Day: Día con prestación del servicio

In [ ]:
# Cargamos el dataset. Usamos un link directo para que el notebook sea reproducible.
# Este CSV tiene un problema de codificación, así que especificamos encoding='latin1'
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv"
df = pd.read_csv(url, encoding='latin1')

# Vistazo inicial
display(df.head())

In [ ]:
# Revisamos la información y tipos de datos
df.info()

In [ ]:
df.describe()

In [ ]:
df['Seasons'].value_counts()

In [ ]:
df['Holiday'].value_counts()

In [ ]:
df['Functioning Day'].value_counts()

### 1.3. Limpieza y Preparación de Variables (Preprocessing)

1.  **Convertir la fecha:** La columna `Date` es un `object` (texto). La convertiremos a formato `datetime` para extraer el mes y el día de la semana.
2.  **Manejar Categóricas:** Las columnas `Seasons`, `Holiday` y `Functioning Day` son categóricas y `scikit-learn` no las entiende como texto. Usaremos **One-Hot Encoding**.
3.  **Definir X e y:** Separar nuestras variables predictoras (X) de nuestra variable objetivo (y).

In [ ]:
# 1. Convertir la fecha
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
# 2. Extraer nuevas características de la fecha
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek # 0=Lunes, 6=Domingo
df.sample(5)

In [ ]:
# 3. Definir variable objetivo (y) y predictoras (X)
y = df['Rented Bike Count']
y.head()

In [ ]:
# Quitamos la variable objetivo y la fecha original (que ya no necesitamos)
X = df.drop(columns=['Rented Bike Count', 'Date'])
X.head()

### 1.4. Definir el `Pipeline` de Preprocesamiento

Para manejar correctamente las variables numéricas (que deben ser escaladas) y las categóricas (que deben ser codificadas), usamos un `ColumnTransformer`. Esto asegura que el escalamiento solo se aplique a los números y el One-Hot-Encoding solo a las categorías.

In [ ]:
# Identificar columnas numéricas y categóricas
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.drop('Hour') # Hour es más como una categoría
categorical_features = X.select_dtypes(include=['object']).columns.tolist() + ['Hour', 'Month', 'DayOfWeek'] # Tratamos Hora, Mes y Día como categóricas

print(f"Columnas Numéricas ({len(numerical_features)}): {numerical_features.tolist()}")
print(f"Columnas Categóricas ({len(categorical_features)}): {categorical_features}")

# Crear transformadores
# 1. Para variables numéricas: Estandarizar (media 0, varianza 1)
numeric_transformer = StandardScaler()

# 2. Para variables categóricas: One-Hot Encoding
categorical_transformer = OneHotEncoder(handle_unknown='ignore') # Ignora categorías no vistas en el test set

# Combinar transformadores con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

### 1.5. División de Datos (Train/Test Split)

Separamos nuestros datos en entrenamiento y prueba. Es crucial que hagamos esto *antes* de la validación cruzada.

In [ ]:
# Dividir en entrenamiento (80%) y prueba (20%)
# En regresión, 'stratify' no se usa. Usamos random_state para reproducibilidad.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

## 2. K-Nearest Neighbors (KNN): "Los similares se comportan de forma similar"

La idea de KNN es simple: para clasificar un nuevo punto u observación, buscamos en los datos de entrenamiento los **'k' puntos más cercanos**. La clasificación o el valor de la variable objetivo del nuevo punto será el de la mayoría de sus 'k' vecinos.

¿Cómo funciona?

1.  Encuentra los `k` puntos más similares en los datos de entrenamiento (basado en las variables predictoras).
2.  **Calcula el promedio** de la variable objetivo.
3.  Ese promedio es la predicción.


Link video explicativo: https://www.youtube.com/watch?v=z3140gOhuVI

**IMPORTANTE:** KNN se basa en medir distancias, por tanto, es obligatorio estandarizar o normalizar las variables predictoras o independientes.



Vamos a construir un primer modelo con un valor arbitrario de `k`, por ejemplo, `k=5`.

In [ ]:
# Es crucial usar un `Pipeline` para que el preprocesamiento (escalado y encoding) se aplique correctamente
# antes de que el modelo KNN calcule las distancias.

# 1. Crear el pipeline completo: (Preprocessor + Modelo)
knn_pipeline_base = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5)) # Usamos k=5 como base
])

# 2. Entrenar el pipeline
knn_pipeline_base.fit(X_train, y_train)

# 3. Realizar predicciones en Train y Test
y_pred_knn_base = knn_pipeline_base.predict(X_test)

# 4. Evaluar el rendimiento
rmse_knn_base = np.sqrt(mean_squared_error(y_test, y_pred_knn_base))
r2_knn_base = r2_score(y_test, y_pred_knn_base)

print("--- KNN Regressor Base (k=5) ---")
print(f"RMSE test: {rmse_knn_base:.2f} bicicletas")
print(f"R-cuadrado test: {r2_knn_base:.4f}")

**Interpretación:**
* **R-cuadrado (R²):** Nos dice que nuestro modelo base explica aproximadamente el **79%** de la variabilidad en la demanda de bicicletas. Pero... ¿es `k=5` el valor óptimo? Y, ¿es 0.79 un valor real o ese valor depende de nuestra división de datos?

## 3. Árboles de Decisión

Un árbol de decisión aprende una serie de reglas `if/else` para segmentar los datos y hacer una predicción. Su gran ventaja es que podemos visualizarlo y entender exactamente cómo toma sus decisiones, lo que hace que su mayor fortaleza sea la interpretabilidad.

¿Cómo funciona?

El objetivo del árbol es encontrar la mejor pregunta (una partición) para dividir un grupo grande y ruidoso en dos nuevos subgrupos que sean lo más homogéneos (o "puros") posible internamente, es decir que los valores dentro del grupo tengan baja varianza.

De esta forma, va dividiendo los datos con reglas, hasta que ya no hay más divisiones (se llega a una hoja final).

Cuando un nuevo dato llega a una hoja final, la predicción es el **promedio** de la variable objetivo de todas las observaciones que cayeron en esa misma hoja durante el entrenamiento.

Link video explicativo: https://www.youtube.com/watch?v=z5rmY-LV7ME

In [ ]:
# 1. Crear el pipeline para el Árbol de Decisión
# No ponemos restricciones (ej. max_depth=None) para ver el sobreajuste
tree_pipeline_base = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

# 2. Entrenar el pipeline
tree_pipeline_base.fit(X_train, y_train)

# 3. Realizar predicciones en el Test set
y_pred_tree_base = tree_pipeline_base.predict(X_test)

# 4. Evaluar el rendimiento
rmse_tree_base = np.sqrt(mean_squared_error(y_test, y_pred_tree_base))
r2_tree_base = r2_score(y_test, y_pred_tree_base)

print("--- Decision Tree Regressor Base (sin podar) ---")
print(f"RMSE test: {rmse_tree_base:.2f} bicicletas")
print(f"R-cuadrado test: {r2_tree_base:.4f}")

**Interpretación:** El R² es de 0.82. El árbol sin podar parece memorizar patrones muy específicos (sobreajuste), lo que le da un rendimiento decente en este caso. Pero de nuevo, ¿tuvimos suerte?

Visualicemos sus reglas de negocio.

In [ ]:
# Visualizar el árbol (solo las primeras capas para que sea legible)

# Necesitamos los nombres de las características DESPUÉS del preprocesamiento
# Extraemos los nombres del preprocesador DENTRO del pipeline ya entrenado
feature_names = tree_pipeline_base.named_steps['preprocessor'].get_feature_names_out()

plt.figure(figsize=(25, 12))
plot_tree(
    tree_pipeline_base.named_steps['model'], # Extraemos el modelo del pipeline
    max_depth=3,
    feature_names=feature_names,
    filled=True,
    fontsize=10,
    precision=1, # Decimales para mostrar en 'value'
    rounded=True
)
plt.title("Visualización Árbol de Regresión (primeras 3 capas)")
plt.show()

**Actividad Práctica: Traducir una Regla**

Miremos la visualización:
* **Nodo Raíz:** La primera división es `num__Temprerature <= -0.1`. Esto prácticamente separa la temperatura en bajo cero y sobre cero.
* **Valor (`value`):** Observen que el `value` en cada nodo es el **promedio de `Rented Bike Count`**. El promedio general (en la raíz) es de 704.8 bicicletas.

**Regla de Negocio (Rama izquierda):**

"Si la temperatura es menor a -0.1 grados y no es otoño y la tempereatura es menor a -0,8 grados y no son las 8 de la mañana, entonces la demanda promedio predicha es de 211 bicicletas.

Esto tiene perfecto sentido económico: en días fríos, fuera de las horas pico, la demanda es baja.

## 4. Evaluando la Robustez con Validación Cruzada (`cross_val_score`)

Hasta ahora, hemos evaluado nuestros modelos en *una sola* división de prueba (el 20% que separamos al inicio). ¿Qué pasa si esa división fue particularmente "fácil" o "difícil" por pura suerte?

La **Validación Cruzada** resuelve esto. En lugar de una sola división, dividimos nuestro `X_train` en (por ejemplo) 5 "folds" (pliegues).

El proceso es:
1.  Entrenar en los pliegues 1, 2, 3, 4 y probar en el 5.
2.  Entrenar en los pliegues 1, 2, 3, 5 y probar en el 4.
3.  ...y así sucesivamente.

Al final, obtenemos 5 scores (uno por cada pliegue). El **promedio** de estos scores es una medida mucho más robusta y confiable del rendimiento real del modelo.

La herramienta para hacer esto rápidamente es `cross_val_score`.

![Validacion_Cruzada](https://drive.google.com/uc?id=1D7-4PHpfdOK3IQ4PIEQqR8WHEafhLO0g)

In [ ]:
print("--- Iniciando Validación Cruzada para KNN Base (k=5) ---")

# 1. Usamos cross_val_score en nuestro pipeline base de KNN
# Lo corremos sobre TODOS los datos de ENTRENAMIENTO (X_train, y_train)
# cv=5 significa 5 folds
# scoring='r2' es nuestra métrica de interés
knn_cv_scores = cross_val_score(
    knn_pipeline_base, # Nuestro modelo base con k=5
    X_train,
    y_train,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

print(f"Scores de cada fold (KNN): {knn_cv_scores}")
print(f"R² Promedio (Robusto): {knn_cv_scores.mean():.4f}")
print(f"R² Desv. Estándar: {knn_cv_scores.std():.4f}")

In [ ]:
print("--- Iniciando Validación Cruzada para Árbol Base (sin podar) ---")

# 2. Repetimos para nuestro pipeline base de Árbol de Decisión
tree_cv_scores = cross_val_score(
    tree_pipeline_base, # Nuestro modelo base sin podar
    X_train,
    y_train,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

print(f"Scores de cada fold (Árbol): {tree_cv_scores}")
print(f"R² Promedio (Robusto): {tree_cv_scores.mean():.4f}")
print(f"R² Desv. Estándar: {tree_cv_scores.std():.4f}")

**Recuerda** que cada score se calculó sobre un subconjunto de validación que el modelo no usó para entrenar en esa ronda específica, por eso el promedio de estos scores es una medida robusta y confiable del rendimiento del modelo.

**Interpretación:**
* **KNN (k=5):** El R² robusto es **0.79**, similar que el que obtuvimos en nuestro `test set` inicial. Esto nos dice que el modelo es estable.
* **Árbol (sin podar):** El R² robusto es **0.79**. Es más bajo que el 0.82 que obtuvimos en el `test set`. Esto sugiere que el árbol sin podar es más "inestable" y sensible a cómo se dividen los datos.

Ahora que sabemos cómo *evaluar* de forma robusta, podemos pasar a *optimizar*.

## 5. Búsqueda de Hiperparámetros Óptimos con Búsqueda de Grilla (`GridSearchCV`)

**Hiperparámetro**

Es una "perilla" o un ajuste de configuración que nosotros, como analistas, debemos definir antes de que un modelo de machine learning comience a entrenar.

Es una configuración externa que controla cómo el modelo va a aprender.

Ejemplos:
- El valor de k en KNN
- La profundidad máxima de un Árbol de Decisión
- La tasa de aprendizaje en una red neuronal

**Parámetro**

Es un valor interno que el modelo aprende durante el entrenamiento a partir de los datos.

Ejemplos:
- Los coeficientes (beta) en una regresión lineal
- Las reglas de división ("si Temperatura < 10...") que encuentra un árbol.



Sabemos que `k=5` o un árbol `sin podar` fueron elecciones arbitrarias. ¿Cómo encontramos el mejor `k` o la mejor `max_depth`?

Podríamos hacer un `cross_val_score` a mano para `k=3`, luego para `k=5`, `k=7`... y comparar los promedios. Pero esto es tedioso.

`GridSearchCV` hace exactamente eso: **automatiza la Búsqueda en Grilla (Grid Search) usando Validación Cruzada (Cross-Validation) como método de evaluación.**

Le damos:
1.  Un modelo (pipeline).
2.  Una "grilla" de hiperparámetros para probar (ej. `k: [3, 5, 7, 11]`).

`GridSearchCV` probará *cada* valor de la grilla, le hará una Validación Cruzada (ej. `cv=5`), calculará el score promedio, y al final nos dirá cuál hiperparámetro tuvo el mejor promedio.

### 5.1. Encontrando el `k` Óptimo para KNN Regressor

In [ ]:
# 1. Re-creamos el pipeline (esto es buena práctica)
knn_pipe_cv = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor())
])

# 2. Definir la grilla de hiperparámetros a probar
# OJO: Usamos 'model__n_neighbors' porque el modelo se llama 'model' dentro del pipeline
param_grid_knn = {
    'model__n_neighbors': [3, 5, 7, 11, 15] # Probaremos estos valores de k
}

# 3. Configurar GridSearchCV
grid_knn = GridSearchCV(
    knn_pipe_cv,
    param_grid_knn,
    cv=5, # 5 folds de validación cruzada
    scoring='r2', # Métrica a optimizar: R-cuadrado
    n_jobs=-1, # Usar todos los procesadores
    verbose=1
)

# 4. Ejecutar la búsqueda (sobre los datos de ENTRENAMIENTO)
print("Iniciando GridSearchCV para KNN...")
grid_knn.fit(X_train, y_train)
print("GridSearchCV para KNN completado.")

# 5. Mostrar los resultados
print(f"\nMejor hiperparámetro 'k': {grid_knn.best_params_}")
print(f"Mejor score R² de validación cruzada: {grid_knn.best_score_:.4f}")

**Visualicemos los resultados** para entender cómo al cambiar `k` cambia el rendimiento.

In [ ]:
# Extraer los resultados de la búsqueda
results_knn = pd.DataFrame(grid_knn.cv_results_)

plt.figure(figsize=(12, 6))
plt.plot(results_knn['param_model__n_neighbors'], results_knn['mean_test_score'], marker='o')
plt.title("Rendimiento de KNN vs. Valor de 'k'")
plt.xlabel("k (Número de Vecinos)")
plt.ylabel("Accuracy Promedio en Validación Cruzada")
plt.xticks(np.arange(3, 30, 2))
plt.show()

### 5.2. Encontrando la Profundidad Óptima para el Árbol de Decisión

In [ ]:
# 1. Re-creamos el pipeline
tree_pipe_cv = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

# 2. Definir la grilla de hiperparámetros
param_grid_tree = {
    'model__max_depth': [3, 5, 7, 10, 15],       # Qué tan profundo puede crecer
    'model__min_samples_leaf': [10, 20, 50, 100] # Mínimo de muestras en una hoja
}

# 3. Configurar GridSearchCV
grid_tree = GridSearchCV(
    tree_pipe_cv,
    param_grid_tree,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

# 4. Ejecutar la búsqueda
print("Iniciando GridSearchCV para Árbol de Decisión...")
grid_tree.fit(X_train, y_train)
print("GridSearchCV para Árbol de Decisión completado.")

# 5. Mostrar los resultados
print(f"\nMejores hiperparámetros: {grid_tree.best_params_}")
print(f"Mejor score R² de validación cruzada: {grid_tree.best_score_:.4f}")

**Interpretación de la Búsqueda:**
* **KNN:** El R² óptimo (alrededor de 0.763) se encontró con `k=5`. Nuestro `k` base estaba cerca.
* **Árbol:** El R² óptimo (alrededor de 0.81) se encontró con `max_depth=10` y `min_samples_leaf=10`. Esto es mucho mejor que el KNN y nos da un modelo "podado" que generaliza mejor que el árbol sin restricciones.

**Nota:** GridSearchCV tiene un costo computacional alto, por lo que no se usa para Big Data. En ese caso se divide el conjunto de datos en 3 partes:
- Train Set (ej. 70%): Para entrenar el modelo.
- Validation Set (ej. 15%): Para ajustar los hiperparámetros.
- Test Set (ej. 15%): Para dar el reporte final de rendimiento sobre datos nuevos.

## 6. Evaluación Final y Variables Relevantes

### 6.1. Evaluación Final en el `Test Set`

Ahora usamos nuestros modelos optimizados (los `best_estimator_` de GridSearchCV) para hacer predicciones finales en el `test set`, nuestro set de datos que hemos guardado hasta ahora.

In [ ]:
# Obtener los mejores modelos encontrados
best_knn_model = grid_knn.best_estimator_
best_tree_model = grid_tree.best_estimator_

# Predecir en el Test set
y_pred_knn_final = best_knn_model.predict(X_test)
y_pred_tree_final = best_tree_model.predict(X_test)

# Calcular R² finales
r2_knn_final = r2_score(y_test, y_pred_knn_final)
r2_tree_final = r2_score(y_test, y_pred_tree_final)

print("--- Comparación de Rendimiento Final (R²) en Test Set ---")
print(f"R² KNN Base (k=5) en Test:     {r2_knn_base:.4f}")
print(f"R² KNN Óptimo (k=5) en Test:   {r2_knn_final:.4f}\n")

print(f"R² Árbol Base (sin podar) en Test:  {r2_tree_base:.4f}")
print(f"R² Árbol Óptimo (podado) en Test: {r2_tree_final:.4f}")

**Conclusión:** ¡El Árbol de Decisión optimizado es nuestro ganador! Con un R² de **~0.81**, explica el 81% de la variabilidad de la demanda de bicicletas en datos nunca antes vistos. La optimización mejoró su rendimiento significativamente.


### 6.2. ¿Cuáles son las Variables Más Relevantes?

Saber qué modelo predice mejor es bueno. Saber qué variables usa es lo que permite tomar **decisiones de negocio**.

#### Para Árboles de Decisión (Fácil)
Los árboles calculan la importancia de una variable (`feature_importance_`) midiendo cuánto reduce la impureza (o el error, en este caso) cada vez que se usa esa variable para una división. Es un subproducto directo de su entrenamiento.

In [ ]:
# 1. Extraer el modelo de árbol del pipeline
tree_model_final = best_tree_model.named_steps['model']

# 2. Extraer los nombres de las características del preprocesador
feature_names = best_tree_model.named_steps['preprocessor'].get_feature_names_out()

# 3. Obtener las importancias
importances = tree_model_final.feature_importances_

# 4. Combinar en un DataFrame y mostrar las 15 más importantes
tree_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("--- Importancia de Variables (Árbol de Decisión) ---")
display(tree_importance_df.head(15))

# 5. Graficar
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=tree_importance_df.head(15))
plt.title('Top 15 Variables Más Importantes (Árbol de Decisión)')
plt.show()

**Interpretación Económica (Árbol):**
El modelo nos dice que los predictores más potentes de la demanda son:
1.  **`num__Temperature(C)`:** La temperatura es, por lejos, el factor más importante.
2.  **`num__Humidity(%)`:** La humedad también es un factor clave.
3.  **`cat__Functioning Day_Yes`:** Si el servicio está operativo o no (¡obvio, pero confirma que el modelo tiene sentido!)
4.  **`cat__Hour_18`:** La hora pico de las 6 PM.

La administración de la ciudad debería centrar sus modelos de predicción y operaciones en el **clima** y las **horas pico**.

#### Para KNN (Más Complejo: Usando `Permutation Importance`)

KNN no tiene un atributo `.feature_importances_`. Su lógica no se basa en coeficientes o cortes, sino en distancias. La "importancia" es implícita.

Para saber qué variables le importan a *cualquier* modelo (incluido KNN), usamos la **Importancia por Permutación**.

**Intuición:**
1.  Calculamos el R² del modelo en el `test set` (ya lo tenemos: ~0.79).
2.  Tomamos *una sola columna* (ej. `Temperatura`) y "barajamos" (permutamos) sus valores aleatoriamente, rompiendo su relación con la demanda de bicicletas.
3.  Volvemos a calcular el R² con esta columna "rota".
4.  Si el R² cae **mucho**, significa que el modelo dependía fuertemente de esa variable. Esa variable es **importante**.
5.  Repetimos esto para todas las variables.

In [ ]:
print("Calculando Importancia por Permutación para KNN (puede tardar un momento)...")

# 1. Usamos permutation_importance en el modelo entrenado y el test set
# NOTA: Le pasamos el pipeline (best_knn_model) y los datos en crudo (X_test)
# El pipeline se encargará de preprocesar los datos internamente.
perm_importance = permutation_importance(
    best_knn_model,
    X_test,
    y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1,
    scoring='r2'
)

# 2. Extraer los nombres de las características
# (Usamos los nombres de las columnas ORIGINALES, ya que permutation_importance
# con un pipeline prueba la importancia de las características de entrada)
feature_names = X_test.columns

# 3. Combinar en un DataFrame
knn_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance_mean': perm_importance.importances_mean, # Caída promedio del R²
    'Importance_std': perm_importance.importances_std
}).sort_values(by='Importance_mean', ascending=False)

print("\n--- Importancia de Variables (KNN por Permutación) ---")
display(knn_importance_df.head(15))

# 4. Graficar
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance_mean', y='Feature', data=knn_importance_df.head(15))
plt.title('Top 15 Variables Más Importantes (KNN por Permutación)')
plt.xlabel('Caída promedio en R² (Importancia)')
plt.show()

## 7. Conclusiones de la Clase

1.  **Modelos Intuitivos para Regresión:** KNN y los Árboles de Decisión se adaptan perfectamente a problemas de regresión. En lugar de "votar" por una clase, **promedian** el valor de sus vecinos (KNN) o de los miembros de su hoja (Árboles).

2.  **CV vs. GridSearch:** Hemos separado dos conceptos clave:
    * **Validación Cruzada (`cross_val_score`)** es una técnica para *evaluar* robustamente un modelo con hiperparámetros fijos (ej. ¿qué tan bueno es mi modelo con k=5?).
    * **Búsqueda de Grilla (`GridSearchCV`)** es una técnica para *optimizar* hiperparámetros, que *usa* la validación cruzada internamente para comparar cuál es el mejor.

3.  **Preprocesamiento es Clave:** Vimos cómo un `Pipeline` y un `ColumnTransformer` son esenciales para manejar de forma robusta las variables numéricas (escalado) y categóricas (one-hot encoding), especialmente para modelos sensibles a la distancia como KNN.

4.  **Importancia de las Variables:** Aprendimos dos formas de obtenerla:
    * **Directa (Árboles):** Usando el atributo `.feature_importances_`.
    * **Agnóstica (Para KNN y otros):** Usando `permutation_importance`, que mide el impacto en el rendimiento del modelo al "romper" una variable.